# Sentiment Classification

### Task
* 네이버에서 영화평을 가지고 positive/negative인지 구분해보자.
* 데이터 불러오기를 제외한 딥러닝 트레이닝 과정을 직접 구현해보는 것이 목표 입니다.

### Dataset
* [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc/)

### Base code
* Dataset: train, val, test로 split
* Input data shape: (`batch_size`, `max_sequence_length`)
* Output data shape: (`batch_size`, 1)
* Architecture:
  * RNN을 이용한 간단한 classification 모델 가이드
  * `Embedding` - `SimpleRNN` - `Dense (with Sigmoid)`
* Training
* Evaluation

### Try some techniques
* Training-epochs 조절
* Change model architectures (Custom model)
  * Use another cells (LSTM, GRU, etc.)
  * Use dropout layers
* Embedding size 조절
  * 또는 one-hot vector로 학습
* Number of words in the vocabulary 변화
* `pad` 옵션 변화
* Data augmentation (if possible)

## Import modules

In [ ]:
use_colab = True
assert use_colab in [True, False]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchtext
!pip install portalocker>=2.0.0
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import time
import shutil
import tarfile

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
import urllib.request

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

import sentencepiece as spm

from collections import Counter, defaultdict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Load Data

* ratings_train.txt: 훈련용으로 사용되는 15만 개의 리뷰
* ratings_test.txt: 테스트용으로 보류된 5만 개의 리뷰
* 모든 리뷰는 140자 이내입니다
* 각 감정 클래스는 동등하게 샘플링되었습니다 (즉, 무작위 추측은 50%의 정확도를 보입니다)
* 10만 개의 부정적 리뷰 (원래 1-4점의 리뷰)
* 10만 개의 긍정적 리뷰 (원래 9-10점의 리뷰)
* 중립적 리뷰 (원래 5-8점의 리뷰)는 제외되었습니다


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7a43fd21d900>)

In [ ]:
train_data = pd.read_table('ratings_train.txt')
train_data = train_data.dropna()
test_data = pd.read_table('ratings_test.txt')
test_data = test_data.dropna()

In [ ]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### Tokenizing


In [ ]:
sp = spm.SentencePieceProcessor()
sp.load('/content/drive/MyDrive/dataset/spm/kowiki.model')  # 모델 경로 설정

# 토크나이저 함수 정의
def tokenizer(text):
    return sp.encode_as_pieces(text)

In [ ]:
for i, (line) in enumerate(train_data['document']):
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    if i == 5:
        break

아 더빙.. 진짜 짜증나네요 목소리
['▁아', '▁더', '빙', '..', '▁진', '짜', '▁', '짜', '증', '나', '네', '요', '▁목', '소', '리']
[26, 228, 4365, 1920, 132, 4351, 3587, 4351, 3922, 3628, 3857, 3760, 266, 3678, 3614]
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['▁', '흠', '...', '포', '스터', '보', '고', '▁초', '딩', '영화', '줄', '..', '..', '오', '버', '연', '기', '조', '차', '▁가', '볍', '지', '▁않', '구', '나']
[3587, 4905, 3344, 3745, 774, 3666, 3600, 194, 4309, 2275, 4081, 1920, 1920, 3686, 3802, 3699, 3605, 3667, 3741, 22, 5279, 3601, 101, 3644, 3628]
너무재밓었다그래서보는것을추천한다
['▁너무', '재', '밓', '었다', '그', '래', '서', '보', '는', '것', '을', '추', '천', '한다']
[2853, 3720, 1, 17, 3630, 3799, 3602, 3666, 3593, 3702, 3598, 3850, 3820, 69]
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['▁교', '도', '소', '▁이야기', '구', '먼', '▁', '..', '솔', '직', '히', '▁재', '미', '는', '▁없다', '..', '평', '점', '▁조정']
[110, 3617, 3678, 2297, 3644, 4086, 3587, 1920, 4240, 3793, 3783, 175, 3680, 3593, 636, 1920, 3851, 3807, 3337]
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['▁사이', '몬', '페'

### Padding and truncating data using pad sequences
* 전부 길이가 다른 리뷰들의 길이를 통일해주자

#### Padding data 출력

### Dataset 구성

## Build the model
### Embedding layer

* embedding-layer는 전체 vocabulary의 갯수(num_words)로 이루어진 index가 `embedding_size`의 *dense vector* 로 변환되는 과정이다.

## Setup hyper-parameters

## Train the model

### Model Evaluate